In [1]:
import numpy as np
import pylab as plt
import matplotlib
from scipy.signal import welch
import seaborn as sns
import types
import time
import pandas as pd
import datetime

from scipy import special
from scipy.stats import norm
from scipy.integrate import solve_ivp

plt.style.use('ggplot')

# Utility function definitions

In [2]:
# sum the oscillators into one signal
# phiMatrix = phases
# A = amplitudes
def sumOsc(phiMatrix, A):
    A = np.resize(A, (1, len(A)))
    return A.dot(np.sin(phiMatrix)).sum(0)

# ODE function and Jacobian

In [34]:
def kernel(r):
    p = special.hermite(4, monic=False)
    return .25 * p(r/np.sqrt(2)) * norm.pdf(r,0,1)

def indexToCoord(i):
    nOneDimension = int(np.sqrt(nOsc))
    x = i % nOneDimension
    y = np.floor(i/nOneDimension)
    return np.array([x,y])
    
def distance(x0, x1):
    coord0 = indexToCoord(x0)
    coord1 = indexToCoord(x1)
    return np.linalg.norm(coord0 - coord1)

def extendedKernelMatrixGet():
    extendedKernelMatrix = np.zeros(shape=(nOsc, nOsc))
    for i in range(nOsc):
        for j in range(i,nOsc):
            r = distance(i,j)
            extendedKernelMatrix[i,j] = (K/nOsc) * kernel(r)
            extendedKernelMatrix[j,i] = extendedKernelMatrix[i,j]
    return extendedKernelMatrix
    
def sinDiffGet(theta):
    A = np.zeros(shape=(nOsc, nOsc))
    for i in range(nOsc):
        for j in range(i, nOsc):
            A[i,j] = theta[i] - theta[j]
            A[j,i] = -A[i,j]
    A = np.sin(A)
    return A

def cosDiffGet(theta):
    A = np.zeros(shape=(nOsc, nOsc))
    for i in range(nOsc):
        for j in range(i, nOsc):
            A[i,j] = theta[i] - theta[j]
            A[j,i] = -A[i,j]
    A = np.cos(A)
    return A

def kuramoto2d(t, theta):
    sinDiff = sinDiffGet(theta)
    thetaDot = W + np.sum(np.dot(extendedKernelMatrix.transpose(),sinDiff), axis=1)
    return thetaDot

def jac(t, theta):
    cosDiff = cosDiffGet(theta)
    return np.sum(np.dot(extendedKernelMatrix.transpose(), cosDiff), axis=1)

# Set the parameters for the spatial Kuramoto model.

In [17]:
# set these simulation parameters
nOsc = 100
upperTimeBound = 10
K = 20

# theta0, W are initial phase, intrinsic freq
theta0 = np.random.uniform(low=0.0, high=2*np.pi, size=nOsc)
W = np.random.normal(loc=0, scale=.5, size=nOsc)

extendedKernelMatrix = extendedKernelMatrixGet()

# Numerically solve the model and plot the results.

In [36]:
# time = quadratic in nOsc with nOsc = 400 => 18secs
start = datetime.datetime.now()

result = solve_ivp(kuramoto2d, [0, upperTimeBound], theta0, jac=jac)

print(nOsc)
print(datetime.datetime.now() - start)
print()
print(result)

100
0:00:01.194986

  message: 'The solver successfully reached the end of the integration interval.'
     nfev: 356
     njev: 0
      nlu: 0
      sol: None
   status: 0
  success: True
        t: array([ 0.        ,  0.04399203,  0.14300331,  0.2420146 ,  0.36140458,
        0.52655771,  0.75326244,  0.98487704,  1.21649164,  1.43589649,
        1.64511438,  1.85290308,  2.0595895 ,  2.25613727,  2.52303652,
        2.75915964,  2.99528276,  3.21766878,  3.44391432,  3.67334187,
        3.88886023,  4.09973864,  4.29948786,  4.50316134,  4.73286907,
        4.97610355,  5.22840128,  5.4946052 ,  5.70347833,  5.90998636,
        6.12607251,  6.35768123,  6.59275283,  6.81068827,  7.02158671,
        7.2268469 ,  7.4036871 ,  7.57645165,  7.83456333,  8.06962663,
        8.30468993,  8.48801035,  8.67133077,  8.86871942,  9.09102801,
        9.27169578,  9.45236356,  9.64721909,  9.81051527,  9.97220401,
       10.        ])
 t_events: None
        y: array([[ 2.94130117,  2.83444975,

In [28]:
result.y.shape

(100, 51)

In [ ]:
completeTimeFlag = False

if completeTimeFlag:
    solution_times = T
    ode_fn = kuramoto_ODE_timeDependent_tf
    jacobian_fn = kuramoto_jac_timeDependent_tf
else:
    solution_times=tfp.math.ode.ChosenBySolver(final_time=upperTimeBound)
    ode_fn = kuramoto_ODE_tf
    jacobian_fn = kuramoto_jac_tf


print("Solving ODEs...")
start = time.time()

results = tfp.math.ode.BDF().solve(ode_fn=ode_fn, 
                            initial_time=0, 
                            initial_state=Y0, 
                            solution_times=solution_times, 
                            jacobian_fn=jacobian_fn)
print("Finished.")
print("Seconds elapsed: {}".format(time.time() - start))

odePhi = results.states.numpy().T
times = results.times.numpy()
orderParameterAbs = [np.abs(np.exp(odePhi[:,i] * (0+1j)).sum())  for i in range(odePhi.shape[1])]

In [ ]:
odePhi.shape

In [ ]:
np.save('odePhi',odePhi)

In [ ]:
plt.figure()
plt.plot(times, orderParameterAbs)
plt.ylabel('Order Parameter Magnitude')
plt.xlabel('Time');
plt.title('Order Parameter evolution');

In [ ]:
index = -1

plt.figure()
plt.hist(np.mod(odePhi[:,index], 2*np.pi),100)
plt.xlim([0, 2 * np.pi])
plt.xlabel('Phase');
plt.ylabel('Count');
plt.title('Phase cluster histogram at index: {}'.format(index))
plt.show()

In [ ]:
# Plotting response
if completeTimeFlag:
    oscList = range(4)
    
    angularVelocity = np.diff(odePhi)/dt
    lenPlot = len(oscList)
    plt.figure(figsize=(15,2 * lenPlot))
    for i, osc in enumerate(oscList):
        plt.subplot(lenPlot, 1, 1+i)
        plt.plot(T[:-1], angularVelocity[osc])
        plt.ylabel("$\dot\phi_{%i}$" %(osc))

# Figure 3a.1 Plot the sum of the oscillators

In [ ]:
if completeTimeFlag:
    plt.figure(figsize=(15,10))
    sumSignal = sumOsc(odePhi, A)
    plt.plot(times, sumSignal)
    plt.xlabel('Time')
    plt.ylabel('Signal sum.')
    plt.title("Sum of all oscillators");

In [ ]:
plt.figure()
sumSignal = sumOsc(odePhi, A)
sig = sumSignal[1600:2000]
plt.plot(sig)

# Figure 3a.2

In [ ]:
fig=plt.figure(figsize=(15,10))
pxx,  freq, t, cax = plt.specgram(sumSignal, Fs=100);
plt.xlabel('Time')
plt.ylabel('Frequency (Hz)')
fig.colorbar(cax).set_label('Intensity [dB]')
plt.title('The spectrum of the global signal as the coupling coefficient changes.');

In [ ]:
f, psd = welch(sumSignal, fs=100)
plt.plot(f,psd);

In [ ]:
parameterList = []
solution_times=tfp.math.ode.ChosenBySolver(final_time=10)
ode_fn = kuramoto_ODE_tf
jacobian_fn = kuramoto_jac_tf
    
for k in np.linspace(1,100,1001):
    K_tf = tf.constant(k * tf.eye(nOsc, dtype=tf.double))
    results = tfp.math.ode.BDF().solve(ode_fn=ode_fn, 
                                initial_time=0, 
                                initial_state=Y0, 
                                solution_times=solution_times, 
                                jacobian_fn=jacobian_fn)
    
    odePhi = results.states.numpy().T
    finalOrderParamter = np.abs(np.exp(odePhi[:,-1] * (0+1j)).sum())
    parameterList.append(finalOrderParamter)

# Figure 3b

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(np.linspace(1,100,1001),parameterList)
plt.xlabel('Coupling parameter K')
plt.ylabel('Magnitude of order parameter at equilibrium (if equilibrium exists)')
plt.title('Phase transition in synchrony by coupling.');

In [ ]:
orderParameterMatrix = []
solution_times=tfp.math.ode.ChosenBySolver(final_time=10)
ode_fn = kuramoto_ODE_tf
jacobian_fn = kuramoto_jac_tf
    
for sigma in np.linspace(1,10,91):
    print(sigma)
    newRow = []
    W_tf = tf.constant(2 * np.pi * np.random.normal(loc=20, scale=sigma, size=nOsc), dtype=tf.double)
    for k in range(1,101):
        K_tf = tf.constant(k * tf.eye(nOsc, dtype=tf.double))
        results = tfp.math.ode.BDF().solve(ode_fn=ode_fn, 
                                    initial_time=0, 
                                    initial_state=Y0, 
                                    solution_times=solution_times, 
                                    jacobian_fn=jacobian_fn)

        odePhi = results.states.numpy().T
        finalOrderParamter = np.abs(np.exp(odePhi[:,-1] * (0+1j)).sum())
        newRow.append(finalOrderParamter)
    orderParameterMatrix.append(newRow)
    
orderParameterMatrix = np.array(orderParameterMatrix)

# Figure 3c

In [ ]:
plt.figure(figsize=(15,10))
plt.imshow(orderParameterMatrix, aspect='auto', origin='lower')
plt.xlabel('Coupling parameter K')
plt.ylabel('Standard deviation of initial frequencies')
plt.colorbar().set_label('Magnitude of order parameter')
plt.title('Coupling/Dispersion relationship for 200 oscillators.  Mean frequency is 20Hz');

# Scratch